## Setting Spark Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Linear_Regression").getOrCreate()

## Importing Linear Regression Library

In [3]:
from pyspark.ml.regression import LinearRegression

## Reading data file in dataframe

In [4]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema = True, header = True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [9]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [11]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [12]:
for x in data.head(1)[0]:
    print(x)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [14]:
for x in data.head(2)[1]:
    print(x)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


## Setting up data in ML format

In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

## Extracting numerical column for Linear regression

In [18]:
assembler = VectorAssembler(inputCols = ["Avg Session Length", "Time on App", "Time on Website", "Length of Membership"],
                            outputCol = "features")

In [19]:
output = assembler.transform(data)

In [21]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [24]:
## Note: features is a DenseVector which is combination of "Avg Session Length" (34.4973), "Time on App" (12.6557), 
##                 "Time on Website" (39.5777), "Length of Membership" (4.0826). 
## Spark needs data for any ML model to be in this format.

## "Yearly Amount Spent" - This is our target.

In [26]:
final_data = output.select("features", "Yearly Amount Spent")

final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Splitting dataset (final_data) into Train, Test

In [27]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                324|
|   mean| 499.41060438377076|
| stddev|  79.72051301838154|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                176|
|   mean|   499.136268800873|
| stddev|    78.788560628432|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



## Creating Linear Regression Model

In [30]:
lr = LinearRegression(featuresCol = "features", labelCol = "Yearly Amount Spent")

In [31]:
lr_model = lr.fit(train_data)

## Prediction

In [32]:
test_result = lr_model.evaluate(test_data)

In [34]:
## Residual value is the difference between Precited value and the actual value

In [33]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  0.286692366511204|
| 11.411892433946718|
|-3.1306892746842436|
| -5.994100601370349|
|-12.731008997680135|
|-21.553895041264354|
| -7.532892320557494|
|  12.26601410610806|
| 3.7395674545088013|
|  4.200506903747851|
| 3.6212529922108274|
| -7.791562728484337|
| -4.865131183028552|
|  -2.18018350282847|
|  5.463621315548437|
|-14.509113010290093|
|-2.1445718030075795|
| 18.754327239093072|
| 7.7322964193171515|
| 1.7181342552940464|
+-------------------+
only showing top 20 rows



## Model Evaluation

In [37]:
#'''Comparing the RootMean Squared Error with the Mean value, Standarad Deviation of the Target. It is concluded that 
# RootMean Squared Error is very low hence the our model is good.'''

test_result.rootMeanSquaredError

10.457516859146756

In [38]:
test_result.r2 # This R2 states our model is explaining 98% variance in the dat which is good.

0.982282377959567

In [39]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## Checking the model accuracy on new data which doesn't have traget in it

In [41]:
unlabeled_data = test_data.select("features")

unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.3091926408918...|
|[31.3662121671876...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6548096756927...|
|[31.7366356860502...|
+--------------------+
only showing top 20 rows



In [42]:
predictions = lr_model.transform(unlabeled_data)

In [43]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.77772139155445|
|[30.7377203726281...|450.36884976228316|
|[30.8794843441274...| 493.3372892595389|
|[31.0613251567161...|493.54955865927195|
|[31.0662181616375...| 461.6643022053545|
|[31.1239743499119...| 508.5009488810301|
|[31.1280900496166...| 564.7855790676122|
|[31.1695067987115...|415.09051669618475|
|[31.3091926408918...| 428.9811503854248|
|[31.3662121671876...|426.38837565273707|
|[31.4459724827577...|481.25571194291774|
|[31.4474464941278...|426.39430482370835|
|[31.5147378578019...|494.67761917948997|
|[31.5171218025062...| 278.0986041532142|
|[31.5316044825729...| 431.0519844138141|
|[31.5741380228732...|  558.918385170877|
|[31.5761319713222...| 543.3711557923359|
|[31.6005122003032...|460.41852425200386|
|[31.6548096756927...|467.53112730823136|
|[31.7366356860502...| 495.2153120002379|
+--------------------+------------